# Encoder/Decoder Dialogue Management

Here we use a simple Encoder/Decoder GRU network to predict answers from the Cornell Movie-Dialog Corpus. We use **PyTorch** as a deep learning framework.

Most of the code in this notebook comes from the following **tutorial** on English-French translation.

https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html

We apply the Machine Translation framework to Dialogue Management by encoding single sentences in the corpus, and decoding their answer.

In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
DEVICE

'cpu'

## Load dataset 

We start loading the corpus' dialogs as **Episodes** (class due.episode.Episode). We limit the number of episodes to load so we can test the code more easily.

In [4]:
from due.corpora import cornell
import itertools

N_DIALOGS = 100

episodes = list(itertools.islice(cornell.episode_generator(), N_DIALOGS))

# episodes = cornell.load()

  0%|          | 0/83097 [00:00<?, ?it/s]


In [5]:
episodes[95].events

[Event(type=<Type.Utterance: 'utterance'>, timestamp=datetime.datetime(2011, 6, 15, 12, 4, 49), agent='u9', payload="What's the worst?"),
 Event(type=<Type.Utterance: 'utterance'>, timestamp=datetime.datetime(2011, 6, 15, 12, 4, 50), agent='u2', payload='You get the girl.')]

## Text cleaning

Here we define functions for a simple text processing pipeline, where we just convert sentences to lowercase and tokenize them using SpaCy.

In [6]:
s = "Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again."

In [7]:
import re
import spacy
# Needed: pipenv run python -m spacy download en
spacy_nlp_en = spacy.load('en')

def tokenize_sentence(sentence):
    s_spacy = spacy_nlp_en(sentence)
    return [str(token) for token in s_spacy]

In [8]:
def normalize_sentence(sentence, return_tokens=False):
    result = sentence.lower()
    result = re.sub(r'\s+', ' ', result)
    result = tokenize_sentence(result)
    if not return_tokens:
        result = ' '.join(result)
    return result

s_normalized = normalize_sentence(s, False)
print(s_normalized)

can we make this quick ? roxanne korrine and andrew barrett are having an incredibly horrendous public break- up on the quad . again .


## Dataset generation

Here we generate a dataset of utterances and their responses. The **output** of this section is:

* A list of utterances (`str`) `X`    
* A list of responses (`str`) `y`, one per utterance in `X`.

Example:

* X: `["hi", "hello how are you?", "i'm fine thanks", ...]`
* y: `["hello how are you?", "i'm fine thanks", "good to hear", ...]`

Note that within an Episode `i`, `y_i` is just `X_i[1:]`. This is not true when `X` and `y` are obtained concatenating data from multiple episodes.

In [9]:
from due.event import Event

In [10]:
def _is_utterance(event):
    return event.type == Event.Type.Utterance

def extract_pairs(episode):
    """
    Process Events in an Episode, extracting all the Event pairs that can be interpreted as one
    dialogue turn (ie. an Agent's utterance, and another Agent's response)
    
    In particular, Event pairs are extracted so that:
    
    * Both Events are Utterances (currently, non-utterances will raise an exception)
    * The second Event immediately follows the first
    * The two Events are acted by two different Agents
    
    Two lists of the same length are returned, so that each utterance (`str`) in the first list
    has its response in the second.
    """
    alternate_sentences = [episode.events[0].payload]
    for e1, e2 in zip(episode.events, episode.events[1:]):
        if not _is_utterance(e1) or not _is_utterance(e2):
            raise NotImplementedError("Non-utterance Events are not supported yet")
            
        if e1.agent != e2.agent:
            alternate_sentences.append(e2.payload)

    normalized_alternate_sentences = [normalize_sentence(s) for s in alternate_sentences]

    result_X = normalized_alternate_sentences[0:-1]
    result_y = normalized_alternate_sentences[1:]
    
    return result_X, result_y
    
extract_pairs(episodes[0])

(['can we make this quick ? roxanne korrine and andrew barrett are having an incredibly horrendous public break- up on the quad . again .',
  "well , i thought we 'd start with pronunciation , if that 's okay with you .",
  'not the hacking and gagging and spitting part . please .'],
 ["well , i thought we 'd start with pronunciation , if that 's okay with you .",
  'not the hacking and gagging and spitting part . please .',
  "okay ... then how 'bout we try out some french cuisine . saturday ? night ?"])

In [11]:
from tqdm import tqdm

X = []
y = []

for e in tqdm(episodes):
    try:
        episode_X, episode_y = extract_pairs(e)
    except AttributeError:
        print("Skipping episode with events: %s" % e.events)
    X.extend(episode_X)
    y.extend(episode_y)

100%|██████████| 100/100 [00:03<00:00, 25.87it/s]


# Vocabulary

Here we index all the words in the corpus so that we can associate each word with a numeric ID, and vice versa.

**TODO**: consider using torchtext instead

**TODO**: implement pruning of rare words

In [12]:
from collections import defaultdict

class Vocabulary():
    def __init__(self):
        self.word_to_index = {}
        self.index_to_word = {}
        self.index_to_count = defaultdict(int)
        self.current_index = 1
        
        self.add_word('<UNK>') # Unknown token
        self.add_word('<SOS>') # Start of String
        self.add_word('<EOS>') # End of String
        
    def add_word(self, word):
        """
        Add a new word to the dictionary.
        """
        if word in self.word_to_index:
            index = self.word_to_index[word]
        else:
            index = self.current_index
            self.current_index += 1
            self.word_to_index[word] = index
            self.index_to_word[index] = word
            
        self.index_to_count[index] += 1
        
    def index(self, word):
        """
        Retrieve a word's index in the Vocabulary. Return the index of the <UNK>
        token if not present.
        """
        if word in self.word_to_index:
            return self.word_to_index[word]
        return self.word_to_index['<UNK>']
    
    def word(self, index):
        """
        Return the word corresponding to the given index/
        """
        return self.index_to_word[index]
    
    def size(self):
        return len(self.word_to_index)

In [13]:
vocabulary = Vocabulary()
vocabulary.word_to_index

{'<UNK>': 1, '<SOS>': 2, '<EOS>': 3}

In [14]:
for sentence in set(X + y):
    for word in sentence.split():
        vocabulary.add_word(word)

In [15]:
vocabulary.size()

803

# Embeddings

We could initialize the model's embedding layer with random weights, but we expect better results using pre-trained word embeddings instead. We chose **GloVe** 6B, 300d word vectors for this purpose.

To set these vectors as default embeddings for our network we need to prepare a matrix of `(vocabulary_size, embedding_dim)` elements where the *i*-th row is the embedding vector of the word of index *i* in our vocabulary.

In [17]:
import numpy as np

In [18]:
from due import resource_manager
rm = resource_manager

In [19]:
def get_embedding_matrix(vocabulary):
    with rm.open_resource_file('embeddings.glove6B', 'glove.6B.300d.txt') as f:
        unk_index = vocabulary.index('<UNK>')
        result = np.zeros((vocabulary.size()+1, 300))
        for line in tqdm(f):
            line_split = line.split()
            word = line_split[0]
            index = vocabulary.index(word)
            if index != unk_index:
                vector = [float(x) for x in line_split[1:]]
                result[index,:] = vector
        sos_index = vocabulary.index('<SOS>')
        result[sos_index,:] = np.ones(300)
        return torch.FloatTensor(result, device=DEVICE)

In [46]:
# embedding_matrix = get_embedding_matrix(vocabulary).cuda()
embedding_matrix = get_embedding_matrix(vocabulary)

400000it [00:13, 30004.51it/s]


In [47]:
embedding_matrix

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 1.0000,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  1.0000],
        ...,
        [ 0.6605, -0.3381,  0.3872,  ...,  0.4240, -0.1595, -0.0201],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]], device='cuda:0')

In [21]:
embedding_matrix = torch.FloatTensor(np.zeros((vocabulary.size()+1, 300)), device=DEVICE)
embedding_matrix

tensor([[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]])

# Encoding

Here we define a function to encode a sentence into a Torch tensor of indices

In [22]:
def sentence_to_tensor(sentence):
    sentence_indexes = [vocabulary.index(w) for w in sentence.split()]
    sentence_indexes.append(vocabulary.index('<EOS>'))
    return torch.tensor(sentence_indexes, dtype=torch.long, device=DEVICE).view(-1, 1)

In [23]:
sentence_to_tensor(X[0])

tensor([[ 174],
        [  28],
        [ 368],
        [  58],
        [ 484],
        [  17],
        [ 485],
        [ 486],
        [  27],
        [ 487],
        [ 488],
        [ 162],
        [ 489],
        [ 255],
        [ 490],
        [ 491],
        [ 492],
        [ 493],
        [ 355],
        [ 196],
        [  92],
        [ 494],
        [  10],
        [ 328],
        [  10],
        [   3]])

# Model

The model we used is copied straight from the one presented in the reference tutorial (https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html).

Note that attention is not implemented yet.

In [24]:
class EncoderRNN(nn.Module):
    def __init__(self, vocabulary_size, hidden_size, embedding_size): # isn't vocab size = 1?
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        
#         self.embedding = nn.Embedding(vocabulary_size, embedding_size) # TODO: load pretrained
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=False)
        self.gru = nn.GRU(embedding_size, hidden_size)
        
    def forward(self, input_data, hidden):
        embedded = self.embedding(input_data).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden
    
    def init_hidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=DEVICE)

In [25]:
class DecoderRNN(nn.Module):
    def __init__(self, vocabulary_size, hidden_size, embedding_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        
#         self.embedding = nn.Embedding(vocabulary_size, embedding_size)
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=False)
        self.gru = nn.GRU(embedding_size, hidden_size)
        self.out = nn.Linear(hidden_size, vocabulary_size)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, input_data, hidden):
        output = self.embedding(input_data).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.out(output[0])
        output = self.softmax(output)
        return output, hidden
    
    def init_hidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=DEVICE)

# Training

Here we define a function to process training for a single pair of sentences.

**TODO** implement batch training

In [26]:
import random

In [27]:
TEACHER_FORCING_RATIO = 0.5
MAX_LENGTH = 500

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.init_hidden()
    
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=DEVICE)
    
    loss = 0
    
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]
        
    decoder_input = torch.tensor([[vocabulary.index('<SOS>')]], device=DEVICE)
    decoder_hidden = encoder_hidden
    
#     use_teacher_forcing = True if random.random() < TEACHER_FORCING_RATIO else False
    use_teacher_forcing = True
    
    if use_teacher_forcing:
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]
        
    loss.backward()
    
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    return loss.item() / target_length

## Model initialization

This instantiate a fresh model. You should run this cell **once** before running your training epochs.

In [28]:
from datetime import datetime

LEARNING_RATE = 0.01
VOCABULARY_SIZE = vocabulary.size() + 1
EMBEDDING_SIZE = 300
HIDDEN_SIZE = 512

encoder = EncoderRNN(VOCABULARY_SIZE, HIDDEN_SIZE, EMBEDDING_SIZE).to(DEVICE)
decoder = DecoderRNN(VOCABULARY_SIZE, HIDDEN_SIZE, EMBEDDING_SIZE).to(DEVICE)

encoder_optimizer = optim.SGD(encoder.parameters(), lr=LEARNING_RATE)
decoder_optimizer = optim.SGD(decoder.parameters(), lr=LEARNING_RATE)
criterion = nn.NLLLoss()

epoch = 0

## Epoch
Here we run a training Epoch, that is, we run the whole dataset through the training procedure. This cell can be executed many times to run multiple Epochs (be careful not to re-initialize the model across Epochs: that would reset training to Epoch 1).

In [30]:
PRINT_EVERY = 50

i = 1
tick = datetime.now()
loss_sum = 0.0
for input_sentence, target_sentence in tqdm(zip(X, y)):
    input_tensor = sentence_to_tensor(input_sentence)
    target_tensor = sentence_to_tensor(target_sentence)

    loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
    loss_sum += loss
    if i%PRINT_EVERY == 0:
        print(i, loss_sum/PRINT_EVERY)
        loss_sum = 0.0
    i += 1
tock = datetime.now()

epoch += 1

print(tock-tick)
print(i, loss_sum/PRINT_EVERY)

51it [00:04, 11.74it/s]

50 5.194382742745282


102it [00:09, 11.17it/s]

100 5.129294074950496


149it [00:13, 11.42it/s]

150 5.166470740610549


201it [00:17, 11.34it/s]

200 5.2998507985255126
0:00:17.734081
202 0.09700315475463867


# Evaluation

In [1]:
# TODO

## Testing 

In [64]:
def predict_answer(input_sentence, encoder, decoder):
    result = []
    
    input_tensor = sentence_to_tensor(input_sentence)
    input_length = input_tensor.size(0)
    
    encoder_hidden = encoder.init_hidden()
    for ei in range(input_length):
        _, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)

    decoder_input = torch.tensor([[vocabulary.index('<SOS>')]], device=DEVICE)
    decoder_hidden = encoder_hidden
    
    for di in range(MAX_LENGTH):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        topv, topi = decoder_output.topk(1)
        decoder_input = topi.squeeze().detach()
        
        predicted_index = decoder_input.item()
        
        if predicted_index == vocabulary.index('<EOS>'):
            break
        result.append(vocabulary.word(predicted_index))
    
    return " ".join(result)

In [73]:
predict_answer("what's the meaning of life?'", encoder, decoder)

"it 's not bad !"